In [ ]:
!pip install pyunpack
!pip install patool

In [ ]:
#Importación de librerías necesarias

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from pyunpack import Archive
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
#Importamos las librerías de costumbre
import numpy as np
import pandas as pd
#Instalamos la librería que nos va a permitir bajar de drive
!pip install gdown
import gdown
#Colocamos la URL del archivo
url = 'https://drive.google.com/uc?id=1LHK1UqsOZMlF9lFuMeRApw6P5mkkexnM'
#Colocamos el nombre al archivo donde la información será guardada
Nombre = 'TipoVehiculo.rar'
#Lo descargamos
gdown.download(url,Nombre,quiet=False)

In [ ]:
#Descompresión de las imágenes
os.mkdir('dataset')
Archive('TipoVehiculo.rar').extractall('dataset/')

In [ ]:
#Se cargan los conjustos de imágenes y se reescalan

train_dir = os.path.join('dataset/TipoVehiculo')

BATCH_SIZE = 15
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             label_mode='categorical')



In [ ]:
#Se crea el dataset de validación

train_batches = tf.data.experimental.cardinality(train_dataset)
validation_dataset = train_dataset.take(train_batches // 4)
train_dataset = train_dataset.skip(train_batches // 4)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(train_dataset))

In [ ]:
#Configuración de los conjuntos de datos para mejor el rendimiento en el entrenamiento usando la API tf.data

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)


In [ ]:
#Usar aumento de datos

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
#Cambiar la escala de los valores de píxeles de las imágenes 

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Creación sel modelo base a partir del modelo previamente entrenado MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

#Congelar la base convolucional

base_model.trainable = False

In [ ]:
#Agregar un encabezado de clasificación

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(8,activation='softmax',kernel_initializer='random_normal',bias_initializer='zeros')

In [ ]:
import json

def write_json(dataz, filename): 
	with open(filename,'w') as f: 
	    json.dump(dataz, f, indent=4) 

data = []

with open('history.json', 'w') as file:
    json.dump(data, file, indent=4)

In [ ]:
#Compilar el modelo y entrenarlo

os.mkdir('modelos')
base_learning_rate = 0.0001
history = []
contador = 0

for i in range(1,101):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    mcp_save = ModelCheckpoint('/content/modelos/transfer_learning_TV_'+str(i)+'.hdf5', save_best_only=True, monitor='val_loss', mode='min',save_weights_only=False)
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history.append(model.fit(train_dataset,epochs=100,callbacks=[mcp_save],validation_data=validation_dataset)) 

    with open('history.json') as json_file: 
	    datay = json.load(json_file) 
     
	    temp = datay

	    y = { 'accuracy': history[contador].history['accuracy'],
            'loss': history[contador].history['val_loss']
      }        		     

	    temp.append(y) 
    write_json(datay, 'history.json')  
 
    contador = contador + 1

    del model

In [ ]:
#Visualización de los resultados en gráficas

for i in range(len(history)):
  plt.plot(history[i].history['accuracy'])
plt.title('Exactitud del modelo')
plt.ylabel('Exactitud')
plt.xlabel('Época')
plt.show()
# "Loss"
for i in range(len(history)):
  plt.plot(history[i].history['val_loss'])
plt.title('Métrica de pérdida')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.show()

In [ ]:
#calculo del porcentajes de validación 

best_val_history = []
for i in range(1,101):
    model = load_model('modelos_TV/transfer_learning_TV_'+str(i)+'.hdf5')
    best_val_history.append(model.evaluate(validation_dataset))
    del model

In [ ]:
#Calculo de promedios de validación y pérdida 

prom_loss = 0.0;
prom_acc = 0.0;
for i in range(len(best_val_history)):
    prom_loss = prom_loss + best_val_history[i][0]
    prom_acc = prom_acc + best_val_history[i][1]

prom_loss = prom_loss / len(best_val_history)
prom_acc = prom_acc / len(best_val_history)

print(prom_loss)
print(prom_acc)

In [ ]:
import shutil
shutil.make_archive("modelos_tipo", 'zip', "modelos")